In [44]:
import os
import numpy as np
import json
import pandas as pd
import re

df_list = []

for root, dirs, files in os.walk('/home/pgajo/projects/def-hsajjad/pgajo/natcode-llm-kgc/results_icl=3'):
    for F in files:
        filename = os.path.join(root, F)
        with open(filename, 'r', encoding='utf8') as f:
            data = json.load(f)
        model_name = data[0]['Model'].split('/')[-1]
        entry = {
            'model': model_name.split('_')[0],
            'module': re.search(r"mod=(\w(-\w)?(-\w)?)", model_name).group(1),
            'dataset': model_name.split('_')[2],
            'lang': model_name.split('_')[3],
            'prompt': model_name.split('_')[4],
            'f1_mean': np.mean([el['F1_Score'] for el in data]).round(3),
            'f1_std': np.std([el['F1_Score'] for el in data]).round(3),
            'prec_mean': np.mean([el['Precision'] for el in data]).round(3),
            'prec_std': np.std([el['Precision'] for el in data]).round(3),
            'rec_mean': np.mean([el['Recall'] for el in data]).round(3),
            'rec_std': np.std([el['Recall'] for el in data]).round(3),
        }

        df_list.append(entry)
        
        
df = pd.DataFrame(df_list)
# display(df)
df_grouped = df.groupby(by=['dataset', 'lang', 'prompt', 'module'])['f1_mean'].mean().reset_index()
df_grouped

,dataset,lang,prompt,module,f1_mean
0,ade,code,rationale,k,0.679
1,ade,code,rationale,k-v,0.697
2,ade,code,rationale,q,0.073
3,ade,code,rationale,q-k,0.555
4,ade,code,rationale,q-k-v,0.753
5,ade,code,rationale,q-v,0.602
6,ade,code,rationale,v,0.623
7,ade,natlang,base,k-v,0.773
8,ade,natlang,base,q,0.724
9,ade,natlang,base,q-k-v,0.774
